<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# IBM Watson OpenScale - Onboard models for monitoring using scored training data table and a sample csv

This notebook must be run in the Python 3.9 or 3.10 runtime environment. It requires Watson OpenScale service credentials.

The notebook demonstrates how to onboard a model (which stores its runtime data in a remote DB2 database) for monitoring in IBM Watson OpenScale. Use the notebook to enable quality, drift, fairness and explainability monitoring. Before you can run the notebook, you must have the following resources:

1. Sample CSV file
2. Scored training data table (existing) in IBM DB2
3. Feedback, Payload, Drifted transactions, Explanations Queue and Result tables details (either existing or to be created) in an IBM DB2.

## Contents

1. [Setup](#setup)
2. [Provide path to sample csv file containing training data](#path-to-csv)
3. [Provide Storage Details](#backend-storage)
4. [Provide Table Details](#table-details)
5. [Provide model details](#model_details)
2. [Provide Spark Compute Engine Details](#spark)
5. [Connect to IBM Watson OpenScale Instance](#connect-openscale)
6. [Connect service provider in IBM Watson OpenScale Instance](#create-service-provider)
7. [Onboard model for monitoring in IBM Watson OpenScale Instance](#create-subscription)
9. [Enable services to monitor model](#enable-monitors)

## Setup <a name="setup"></a>

### Installing Required Libraries

First import some of the packages you need to use. After you finish installing the following software packages, restart the kernel.

In [ ]:
import warnings
warnings.filterwarnings("ignore")
%env PIP_DISABLE_PIP_VERSION_CHECK=1

# Note: Restart kernel after the dependencies are installed
!pip install --upgrade ibm-watson-openscale
!pip install "ibm_wos_utils>=4.6.0"

## Provide path to sample csv file containing model input and output including label column <a name="path-to-csv">
This csv file is required to understand model input and output columns and their data-types. Provide path location of csv file here.

Please note if you are executing this notebook in IBM Watson Studio, first upload the csv file to project and use provided code snippet to download it to local directory of this notebook.

In [ ]:
# # Download "sample_csv" from project to local directory
# from ibm_watson_studio_lib import access_project_or_space
# wslib = access_project_or_space()
# wslib.download_file("sample_csv")
sample_csv = "<to_be_edited>"

## Provide Backend Storage Details <a name="backend-storage"></a>

IBM Watson OpenScale services monitors models by analyzing runtime data, i.e., the data model is making predictions on. To do this analysis, most of the services require access to this runtime data (also called payload data). In addition, some of the services may require access to manually labelled runtime data (also called feedback data). Hence, user needs to store such data in some backend storage and connect this storage to IBM Watson OpenScale.

### Provide DB2 connection details

| Parameter | Description | Possible Value(s) |
| :- | :- | :- |
| type | Describes the type of storage being used. For DB2, this must be set to `jdbc`. | `jdbc` |
| jdbc_url | Connection string for jdbc. Example: `jdbc:db2://jdbc_host:jdbc_port/database_name` | |
| jdbc_driver | Class name of the JDBC driver to use to connect. Example: for DB2 use `com.ibm.db2.jcc.DB2Driver` ||
| use_ssl | Boolean value to indicate whether to use SSL while connecting | `True` or `False` |
| certificate | SSL Certificate [Base64 encoded string] of the JDBC Connection. Ignored if `use_ssl` is `False`. |
| location_type | Identifies the type of location for connection to use. For DB2, this must be set to `jdbc`. | `jdbc` |
| username | Username of the JDBC Connection | |
| password | Password of the JDBC Connection | |
| database | Name of database hosting training data table | |
| schema | Name of schema hosting training data table | |
| table | Name of training data table | |

In [ ]:
datawarehouse_details = {
    "type": "jdbc",
    "connection": {
        "jdbc_url": "<to_be_edited>",
        "jdbc_driver": "<to_be_edited>",
        "use_ssl": "<to_be_edited>",
        "certificate": "<to_be_edited>"
    },
    "credentials": {
        "username": "<to_be_edited>",
        "password": "<to_be_edited>"
    }
}

## Provide details of different tables<a name="table-details"></a>

IBM Watson OpenScale services require different tables to perform their analysis. Depending on which services you have enabled, provide details of the corresponding tables.
Tables are:

| Table | Description |
| :- | :- |
| Payload Table | Hosts the runtime data predicted by model. Required for detecting fairness and drift in runtime data. |
| Feedback Table | Hosts the manually labelled runtime data (also called feedback data) predicted by model. Required for tracking quality of monitor by analyzing feedback data. |
| Drifted Transactions Table | Hosts the data identified to be drifted.|
| Explain Queue Table | Hosts the data for which explanations are required to be generated. This can be same as payload table.|
| Explain Results Table | Hosts the explanations generated for records in explain queue table. |
| Scored Training Data Table | Contains the details of table containing scored training data. If you dont have this table available, Please refer to [this notebook](https://github.ibm.com/aiopenscale/api-client-utils/blob/master/notebooks/batch/4.6/jdbc/common_configuration_notebook_simplified_jdbc.ipynb) for creating Scored Training Data Table. Scored training data table should be available in the DATABASE. |

For each of the table, following information is required:

| Parameter | Description |
| :- | :- |
| database | Name of the database hosting the schema. |
| schema | Name of the schema hosting the table. |
| table | Name of the table. |
| auto_create | Boolean value identifying if the table already exists or has to be created via IBM Watson OpenScale. |
| partition_column | The column to help Spark read and write data using multiple workers in your JDBC storage. This will help improve the performance of your Spark jobs. |
| num_partition | An integer value which defines maximum number of partitions that Spark can divide the data into. In JDBC, it also means the maximum number of connections that Spark can make to the JDBC store for reading/writing data. |

In [ ]:
DATABASE_NAME="<to_be_edited>"
SCHEMA_NAME="<to_be_edited>"

# Scored trainin data table information 
scored_training_data_table = { 
    "data": {
        "auto_create": False, #set it to False if table already exists
        "database": DATABASE_NAME,
        "schema": SCHEMA_NAME,
        "table": "<to_be_edited>"
    },
    "parameters":{
        "partition_column": "<to_be_edited>",
        "num_partitions": "<to_be_edited>"
    }
}

In [ ]:
### Recommended number of partitions calculation
# num_partitions_recommended = 12

# if spark_parameters:
#     executors = spark_parameters.get("max_num_executors", 2)
#     cores = spark_parameters.get("executor_cores", 2)
#     num_partitions_recommended = 2 * executors * cores
    
# print("{} is the recommended value for number of partitions in your data. \
# Please change this value as per your data.".format(num_partitions_recommended))

# Payload table information
payload_table = {
    "data": {
        "auto_create": True, #set it to False if table already exists
        "database": DATABASE_NAME,
        "schema": SCHEMA_NAME,
        "table": "<to_be_edited>"
    },
    "parameters":{
        "partition_column": "<to_be_edited>",
        "num_partitions": "<to_be_edited>"
    }
}

# Feedback table information
feedback_table = {
    "data": {
        "auto_create": True, #set it to False if table already exists
        "database": DATABASE_NAME,
        "schema": SCHEMA_NAME,
        "table": "<to_be_edited>"
    },
    "parameters":{
        "partition_column": "<to_be_edited>",
        "num_partitions": "<to_be_edited>"
    }
}

# The below tables are required by monitors

#Drifted Transaction table. 
#Set this table information if drift is enabled
drifted_transaction_table = {
    "data": {
        "auto_create": True, #set it to False if table already exists
        "database": DATABASE_NAME,
        "schema": SCHEMA_NAME,
        "table": "<to_be_edited>"
    },
    "parameters":{
        "partition_column": "<to_be_edited>",
        "num_partitions": "<to_be_edited>"
    }
}

#Explanation Result table
#Set this table information if Explain is enabled
explain_result_table = {
    "data": {
        "auto_create": True, #set it to False if table already exists
        "database": DATABASE_NAME,
        "schema": SCHEMA_NAME,
        "table": "<to_be_edited>"
    }
}

#Explanation Queue table
#Set this table information if Explain is enabled
explain_queue_table = {
    "data": {
        "auto_create": True, #set it to False if table already exists
        "database": DATABASE_NAME,
        "schema": SCHEMA_NAME,
        "table": "<to_be_edited>"
    },
    "parameters":{
        "partition_column": "<to_be_edited>",
        "num_partitions": "<to_be_edited>"
    }
    
}

## Provide Model Details <a name="model-details"></a>

| Parameter | Description | Possible Value(s) |
| :- | :- | :- |
| label_column | The column which contains the target field (also known as label column or the class label). | |
| model_type | Enumeration classifying if your model is a binary or a multi-class classifier or a regressor. | `binary`, `multiclass`, `regression` |
| prediction | The column containing the model output. This should be of the same data type as the label column. | |
| probability | The column (of type array) containing the model probabilities for all the possible prediction outcomes. This is not required for regression models. | |
| url | scoring url for the deployed model.||
| token | scoring token for the deployed model. This is required only for Azure ML studio model||
| feature_columns | Columns identified as features by model. If user is not providing this, it will be inferred from the input csv file.| A list of column names, `None` |
| categorical_columns | Feature columns identified as categorical by model. If user is not providing this, it will be inferred from the input csv file.| A list of column names,  `None` |

## Select IBM Watson OpenScale services

| Parameter | Description | Possible Value(s) |
| :- | :- | :- |
| enable_quality | Boolean value to allow generation of common configuration details needed if quality alone is selected | `True` or `False` |
| enable_fairness | Boolean value to allow generation of fairness specific data distribution needed for configuration | `True` or `False` |
| enable_drift | Boolean value to allow generation of Drift Archive containing relevant information for Model and Data Drift. | `True` or `False` |
| enable_explainability | Boolean value to allow generation of explainability configuration and perturbations | `True` or `False` |
| parameters | Provide the parameters for a monitor that needs to get enabled, |
| thresholds | Provide the thresholds for faireness and quality monitor if that monitor needs to get enabled |
| train_drift_model | It is set to `True` to train drift model and learn stats online. |`True` or `False` |
| enable_online_learning | It is set to `True` to generate the stats and scored perturbations online. |`True` or `False` |

In [ ]:
model_info = {
    "model_type": "<to_be_edited>",
    "label_column": "<to_be_edited>",
    "prediction": "<to_be_edited>",
    "probability": "<to_be_edited>",
    "feature_columns": ["<to_be_edited>"],
    "categorical_columns": ["<to_be_edited>"],
    "scoring":{
        "url":"<to_be_edited>",
        "token":"<to_be_edited>"
    }
}

monitors_config = {
    "fairness_configuration": {
        "enabled": True,
        "parameters":{
        },
        "thresholds": [
        ]
    },
    "quality_configuration": {
        "enabled": True,
        "parameters" : {
        },
        "thresholds" : [
        ]
    },
    "drift_configuration": {
        "enabled": True,
        "parameters":{
            "train_drift_model": True
        }
    },
    "explainability_configuration":{
        "enabled": True,
        "parameters":{
            "enable_online_learning": True
        }
    }
}

## Provide Spark Connection Details <a name="spark"></a>

To generate configuration for monitoring models in IBM Watson OpenScale, a spark compute engine is required. It can be either IBM Analytics Engine or your own Spark Cluster. Provide details of any one of them in this section.

Please note, if you are using your own Spark cluster, checkout IBM Watson OpenScale documentation on how to setup spark manager API to enable interface for use with IBM Watson OpenScale services.

### Parameters for IBM Analytics Engine
If your job is going to run on Spark cluster as part of an IBM Analytics Engine instance on IBM Cloud Pak for Data, enter the following details:

| Parameter | Description | Possible Value(s) |
| :- | :- | :- |
| display_name | Display Name of the Spark instance in IBM Analytics Engine | |
| location_type | Identifies if compute engine is IBM IAE or Remote Spark. For IBM IAE, this must be set to `cpd_iae`. | `cpd_iae` |
| endpoint | Spark Jobs Endpoint for IBM Analytics Engine | |
| volume | IBM Cloud Pak for Data storage volume name | |
| username | IBM Cloud Pak for Data username | |
| apikey | IBM Cloud Pak for Data API key | |

### Parameters for Remote Spark Cluster
If your job is going to run on Spark Cluster as part of a Remote Hadoop Ecosystem, enter the following details:

| Parameter | Description | Possible Value(s) |
| :- | :- | :- |
| location_type | Identifies if compute engine is IBM IAE or Remote Spark. For Remote Spark, this must be set to `custom`. | `custom` |
| endpoint | Endpoint URL where the Spark Manager Application is running | |
| username | Username to connect to Spark Manager Application | |
| password | Password to connect to Spark Manager Application | |


### Provide Spark Resource Settings [Optional]
Configure how much of your Spark Cluster resources can this job consume. Leave the variable `spark_settings` to `{}` if no customisation is required.

| Parameter | Description |
| :- | :- |
| max_num_executors | Maximum Number of executors to launch for this session |
| min_executors | Minimum Number of executors to launch for this session |
| executor_cores | Number of cores to use for each executor |
| executor_memory | Amount of memory (in GBs) to use per executor process |
| driver_cores | Number of cores to use for the driver process |
| driver_memory | Amount of memory (in GBs) to use for the driver process |

In [ ]:
spark_connection_info = {
    "connection": {
        "endpoint": "<to_be_edited>",
        "location_type": "<to_be_edited>",
        "display_name": "<to_be_edited>",
        "volume": "<to_be_edited>",
        "instance_id":"<to_be_edited>"
    },
    "credentials": {
        "username": "<to_be_edited>",
        "password": "<to_be_edited>",
        "apikey": "<to_be_edited>"
    }
}

"""
Example:

spark_settings = {
    # max_num_executors: Maximum Number of executors to launch for this session
    "max_num_executors": "2",
    
    # min_executors: Minimum Number of executors to launch for this session
    "min_executors": "1",
    
    # executor_cores: Number of cores to use for each executor
    "executor_cores": "2",
    
    # executor_memory: Amount of memory (in GBs) to use per executor process
    "executor_memory": "2",
    
    # driver_cores: Number of cores to use for the driver process
    "driver_cores": "2",
    
    # driver_memory: Amount of memory (in GBs) to use for the driver process 
    "driver_memory": "1"
}
"""
spark_settings = {}

spark_connection_info["spark_settings"] = spark_settings

## Connect to IBM Watson OpenScale instance <a name="connect-openscale"></a>

Following information is required to connect to IBM Watson OpenScale instance:

| Parameter | Description |
| :- | :- |
| url | Base url of your Cloud Pak for Data cluster hosting IBM Watson OpenScale instance. |
| username | Username to connect to your IBM Watson OpenScale instance in Cloud Pak for Data cluster. |
| password | Password to connect to your IBM Watson OpenScale instance in  Cloud Pak for Data cluster. One of `password` or `api_key` must be provided. |
| api_key | API Key to connect to your IBM Watson OpenScale instance in Cloud Pak for Data cluster. One of `password` or `api_key` must be provided. |
| service_instance_id | Id of your IBM Watson OpenScale Instance |

In [ ]:
from ibm_cloud_sdk_core.authenticators import CloudPakForDataAuthenticator
from ibm_watson_openscale import APIClient

import warnings
warnings.filterwarnings('ignore')

service_instance_id = "<SERVICE_INSTANCE_ID>" #Default is 00000000-0000-0000-0000-000000000000
service_credentials = {
    "url": "<to_be_edited>",
    "username": "<to_be_edited>",
    "password": "<to_be_edited>"
    #     "apikey":"<to_be_edited>"
}

authenticator = CloudPakForDataAuthenticator(
    url=service_credentials['url'],
    username=service_credentials['username'],
    password=service_credentials['password'],
    #     apikey=service_credentials['apikey'],
    disable_ssl_verification=True
)

client = APIClient(
    service_url=service_credentials['url'],
    service_instance_id=service_instance_id,
    authenticator=authenticator
)

print(client.version)

## Configure Machine Learning Provider in IBM Watson OpenScale instance <a name="create-service-provider"></a>

Before configuring model for monitoring in IBM Watson OpenScale, you need to connect your machine learning provider with IBM Watson OpenScale instance. Since, we are configuring a model for monitoring which has its runtime data located remotely to IBM Watson OpenScale, we'll create a custom machine learning provider in given instance.

Following details are required:

| Parameter | Description |
| :- | :- |
| name | Name of the machine learning provider being configured. This can be any string value. |
| description | Description for the machine learning provider being configured. |
| service_type | Identifies type of the machine learning provider. In this case, this value must be `ServiceTypes.CUSTOM_MACHINE_LEARNING` |
| credentials | Stores username and password to connect to machine learning provider. |
| deployment_space_id | Identifies the space where the model is deployed. |
| operational_space_id | Defines the classification of machine learning provider. Possible values are `pre-production` and `production`. |

In [ ]:
# [OPTIONAL] Delete existing service provider with the same name as provided

# SERVICE_PROVIDER_NAME = "<to_be_edited>"
# service_providers = client.service_providers.list().result.service_providers
# for provider in service_providers:
#     if provider.entity.name == SERVICE_PROVIDER_NAME:
#         client.service_providers.delete(service_provider_id=provider.metadata.id)
#         break

# Add Service Provider
from ibm_watson_openscale.supporting_classes.enums import ServiceTypes
# from ibm_watson_openscale.base_classes.watson_open_scale_v2 import CustomCredentials

added_service_provider_result = client.service_providers.add(
        name="<to_be_edited>",
        description="<to_be_edited>",
        service_type=ServiceTypes.CUSTOM_MACHINE_LEARNING,
        credentials={},
        deployment_space_id = "<to_be_edited>",
        operational_space_id="<to_be_edited>",
        background_mode=False
    ).result

service_provider_id = added_service_provider_result.metadata.id

client.service_providers.show()

## Onboard model for monitoring in IBM Watson OpenScale instance <a name="create-subscription"></a>

When you configure a model for monitoring in IBM Watson OpenScale instance, a corresponding subscription is created for this model. Following details are required:

| Parameter | Description |
| :- | :- |
| subscription_name | Name of the subscription to use. This can be any string value typically identifying model being monitored. |
| datamart_id | Same as id of IBM Watson OpenScale instance. |
| service_provider_id | Id of the machine learning provider instance created in IBM Watson OpenScale. |
| model_info | Details of the model to be monitored |
| sample_csv | Path to the csv file containing scored training data |
| spark_credentials | Connection details of Spark compute engine to use for analysis by different IBM Watson OpenScale services. |
| payload_table | Details of the payload table to be used with this subscription. |
| feedback_table | Details of the feedback table to be used with this subscription. |
| scored_training_data_table | Details of the scored training data table to be used with this subscription. |
| managed_by | To identify whether the subscription is `system` managed (Model transactions are stored in the OpenScale database and evaluated using OpenScale computing resources) or `self` managed (Model transactions are stored in a your own data warehouse and evaluated by your Spark analytics engine.) . This function is not supporting system managed subscriptions as of now. |

In [ ]:
subscription_id = client.subscriptions.create_subscription_using_training_data(
    subscription_name="My SDK Batch Subscription-db2",
    datamart_id=service_instance_id,
    service_provider_id=service_provider_id,
    model_info=model_info,
    sample_csv = sample_csv,
    spark_credentials=spark_connection_info,
    data_warehouse_connection = datawarehouse_details,
    payload_table=payload_table,
    feedback_table=feedback_table,
    scored_training_data_table = scored_training_data_table,
    managed_by="self"
)

# print("Subscription id is {}".format(subscription_id))

# Wait for the subscription to get in active state and to create the 
# required tables in the background before moving onto enabling monitors

# import time
# from datetime import datetime

# subscription_status = None
# while subscription_status not in ("active", "error"):
#     subscription_status = client.subscriptions.get(subscription_id).result.entity.status.state
#     if subscription_status not in ("active", "error"):
#         print(datetime.now().strftime("%H:%M:%S"), subscription_status)
#         time.sleep(15)
        
# print(datetime.now().strftime("%H:%M:%S"), subscription_status)

## Enable different services to monitor model <a name="enable-monitors"></a>

Depending on the services enabled in `monitors_config`, different services are enabled in given subscription. There services are called monitors.

Following details are required:

| Parameter | Description |
| :- | :- |
| datamart_id | Same as id of IBM Watson OpenScale instance. |
| subscription_id | Id of the subscription created for given model in IBM Watson OpenScale instance. |
| monitors_config | Details of the monitores that needs to get configured. |
| drifted_transaction_table | Details of the drifted transactions table to be used with this subscription. |
| explain_queue_table | Details of the explain queue table to be used with this subscription. |
| explain_results_table | Details of the explain results table to be used with this subscription. |

In [ ]:
instance_ids = client.monitor_instances.enable_monitor_using_training_data(
                  datamart_id = service_instance_id,
                  subscription_id = subscription_id,
                  monitors_config = monitors_config,
                  drifted_transaction_table = drifted_transaction_table,
                  explain_queue_table = explain_queue_table,
                  explain_results_table = explain_result_table)

print(instance_ids)

## Track each monitor instance status
# for key, value in instance_ids.items():
#     monitor_instance_status = None

#     while monitor_instance_status not in ("active", "error"):
#         monitor_instance_details = client.monitor_instances.get(monitor_instance_id=value).result
#         monitor_instance_status = monitor_instance_details.entity.status.state
#         if monitor_instance_status not in ("active", "error"):
#             print(datetime.now().strftime("%H:%M:%S"), monitor_instance_status)
#             time.sleep(30)

#     print(key, monitor_instance_status)

## Congratulations!

All the monitors have been enabled. It will take some time for monitors to get into active state. You can track the status of each monitor separately by using above code snippet.

Once, all monitors are active, load data into payload or feedback table and either run on-demand evaluations or wait for scheduled evaluations to complete for each monitor. You can check more details in [Watson OpenScale Dashboard](https://url-to-your-cp4d-cluster/aiopenscale).